In [1]:
%matplotlib inline
%tensorflow_version 1.x
import os
import cv2  
import warnings
import numpy as np
import pandas as pd
import seaborn as sn 

sn.set(font_scale=1.4)
warnings.filterwarnings('ignore')

import tensorflow as tf                
import matplotlib.pyplot as plt

from tqdm import tqdm
from datetime import datetime
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import keras
from keras import metrics
from keras import backend as K
from keras.utils import to_categorical
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Input, BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionResNetV2, ResNet50, Xception, VGG19, VGG16
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint, LambdaCallback, Callback

TensorFlow 1.x selected.


Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip drive/My\ Drive/dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: dataset/Train Images/image1844.jpg  
  inflating: dataset/Train Images/image1845.jpg  
  inflating: dataset/Train Images/image1849.jpg  
  inflating: dataset/Train Images/image1851.jpg  
  inflating: dataset/Train Images/image1852.jpg  
  inflating: dataset/Train Images/image1854.jpg  
  inflating: dataset/Train Images/image1859.jpg  
  inflating: dataset/Train Images/image186.jpg  
  inflating: dataset/Train Images/image1860.jpg  
  inflating: dataset/Train Images/image1861.jpg  
  inflating: dataset/Train Images/image1862.jpg  
  inflating: dataset/Train Images/image1863.jpg  
  inflating: dataset/Train Images/image1865.jpg  
  inflating: dataset/Train Images/image1868.jpg  
  inflating: dataset/Train Images/image187.jpg  
  inflating: dataset/Train Images/image1873.jpg  
  inflating: dataset/Train Images/image1875.jpg  
  inflating: dataset/Train Images/image1876.jpg  
  inflating: dataset/Train Images/image1878.jpg  
 

In [5]:
!ls dataset

 test.csv  'Test Images'   train.csv  'Train Images'


In [0]:
TRAIN_PATH = 'dataset/Train Images/'
TEST_PATH = 'dataset/Test Images/'

IMAGE_SHAPE = (224, 224)
IMAGE_SHAPE_FULL = (224, 224, 3)
RESIZE_FACTOR = 255
EPOCHS = 25
BATCH_SIZE = 8

In [5]:
class_names_label = {'Food': 0, 'Attire': 1, 'Decorationandsignage': 2, 'misc': 3}
class_label_names = {v: k for k, v in class_names_label.items()}
class_names = list(class_names_label.keys())
print(class_label_names)
print(class_names_label)
nb_classes = len(class_names_label)

{0: 'Food', 1: 'Attire', 2: 'Decorationandsignage', 3: 'misc'}
{'Food': 0, 'Attire': 1, 'Decorationandsignage': 2, 'misc': 3}


In [0]:
def load_images(df, path, shape, resize, labels_provided=True):
    images = []
    labels = []
            
    for i in range(len(df.Image)):    
        image = cv2.imread(path + str(df.Image[i]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, shape)
        images.append(image)
        if labels_provided is True:
            labels.append(df.Class[i])
    if labels_provided is True:
        return np.array(images)/resize, labels
    else:
        return np.array(images)/resize

In [7]:
train_ = pd.read_csv('dataset/train.csv')
test_ = pd.read_csv('dataset/test.csv')
train_['Class'] = train_['Class'].map(class_names_label).astype(np.uint8)
train_.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
train_images, train_labels = load_images(train_, TRAIN_PATH, IMAGE_SHAPE, RESIZE_FACTOR)
test_images = load_images(test_, TEST_PATH, IMAGE_SHAPE, RESIZE_FACTOR, labels_provided=False)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train_images, 
                                                    to_categorical(train_labels), 
                                                    test_size=0.3, 
                                                    random_state=42)

In [0]:
# def create_model(input_shape, num_classes): 
    
# #     pretrain_model = ResNet50(weights='imagenet', 
# #                       include_top=False, 
# #                       input_shape=input_shape, 
# #                       pooling='avg')
    
# #     pretrain_model.trainable = False
        
# #     input_tensor = Input(shape=input_shape)
# #     x = pretrain_model(input_tensor)
# #     output = Dense(num_classes, activation='softmax')(x)
# #     model = Model(input_tensor, output)    
    
#     pretrain_model = VGG16(include_top=False, 
#                            weights='imagenet',
#                            input_shape=input_shape,
#                            pooling='avg')
    
#     pretrain_model.trainable = False
        
#     input_tensor = Input(shape=input_shape)
#     x = pretrain_model(input_tensor)
#     x = Dense(256,activation='relu')(x)
#     output = Dense(num_classes, activation='softmax')(x)
#     model = Model(input_tensor, output)    
        
#     return model


def create_model(input_shape, num_classes):
    
    pretrain_model = VGG19(include_top=False, 
                            weights='imagenet', 
                            input_shape=input_shape)    
    pretrain_model.trainable = False
    
    input_tensor = Input(shape=input_shape)
    # bn = BatchNormalization()(input_tensor)
    x = pretrain_model(input_tensor)
    # x = Conv2D(128, kernel_size=(1,1), activation='relu')(x)
    # x = Flatten()(x)
    # x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    # x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

In [15]:
keras.backend.clear_session()
model = create_model(input_shape=IMAGE_SHAPE_FULL, num_classes=nb_classes)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
dense_1 (Dense)              (None, 7, 7, 512)         262656    
_________________________________________________________________
dense_2 (Dense)              (None, 7, 7, 4)           2052      
Total params: 20,289,092
Trainable params: 264,708
Non-trainable params: 20,024,384
_________________________________________________________________


In [0]:
earlystop = EarlyStopping(monitor='val_loss', 
                          patience=3)

reduce_learning_rate = ReduceLROnPlateau(monitor='loss',
                                         factor=0.1,
                                         patience=2,
                                         cooldown=2,
                                         min_lr=0.00001,
                                         verbose=1)

callbacks = [reduce_learning_rate]

In [0]:
datagen = ImageDataGenerator(featurewise_center=False,  # set input mean to 0 over the dataset
                             samplewise_center=False,  # set each sample mean to 0
                             featurewise_std_normalization=False,  # divide inputs by std of the dataset
                             samplewise_std_normalization=False,  # divide each input by its std
                             zca_whitening=False,  # apply ZCA whitening
                             rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
                             zoom_range = 0.1, # Randomly zoom image 
                             width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
                             height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=False)  # randomly flip images

datagen.fit(X_train)

history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE), 
                              validation_data=(X_test, y_test),
                              steps_per_epoch=len(X_train)//BATCH_SIZE, 
                              epochs=EPOCHS, 
                              callbacks=callbacks)

In [0]:
# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(121)
    plt.plot(history.history['accuracy'],'bo--', label = "acc")
    plt.plot(history.history['val_accuracy'], 'ro--', label = "val_acc")
    plt.title("train_accuracy vs val_accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(122)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()
    
plot_accuracy_loss(history)    

In [0]:
pred_labels = model.predict(test_images)
pred_labels = [np.argmax(label) for label in pred_labels]
pred_class_label = [class_label_names[label] for label in pred_labels]

In [0]:
submission = pd.DataFrame({ 'Image': test_.Image, 'Class': pred_class_label })
submission.head()

In [0]:
# Saving Model and Submission Files
now = datetime.now()
model.save('model_' + str(now) + '.h5')
submission.to_csv('drive/My Drive/submission_' + str(now) + '.csv', index=False)